In [35]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['cpDB']
collection = db.test1

In [36]:
timestep = 15 # in minutes

In [37]:
powerLimit = 50

In [41]:
def database_update(_cpid, changes):
    query = {"evse_id":_cpid}
    collection.update_one(query,changes)
    
    
def optimization(powerLimit): #in real situation, _cpGroup shoud be included
    # get updated status from cpDB
    charging_pool =[]
    totalPowerDemand =0
    flex_supply =0
    dbcopy =[]
    global timestep
    
    
    #use pymongo to change status of all finished events, think about how to record flexibility?! the values will be deleted once the EV is driven away 
    
    #collection.update_many({'cp_group':'G1','status':'O',"$or":[{'chargingInfo.E':{"$lte":0}},{'chargingInfo.T':0}]},{"$set":{'status': 'Finished','updatedPower': 0, 'chargingInfo.E': None,'chargingInfo.T': None}})
    collection.update_many({'cp_group':'G1','status':'L'},{"$set":{'status': 'Finished','updatedPower': 0}})
    
    #for the last timestep events, change status, update new power
    for i in collection.find({'cp_group':'G1','status':'O','chargingInfo.T':timestep}):
        
        _updatedPower = min(i['powerValue'],(4* i['chargingInfo']['E']))
        newPowerLevel = {"$set":{"updatedPower":_updatedPower, 'status':'L'}}
        database_update(i['evse_id'], newPowerLevel) 
        powerLimit -= _updatedPower
        
        ####for simulation
        _energy = - _updatedPower*0.25
        _updatedStatus = {"$inc":{"chargingInfo.E":_energy,'chargingInfo.T': -timestep}}
        database_update(i['evse_id'], _updatedStatus) 
    
    for i in collection.find({'cp_group':'G1','status':'O'},{'_id':0,'evse_id':1, 'powerValue':1,'chargingInfo.user_id':1, 'chargingInfo.E':1,'chargingInfo.T':1}):
        dbcopy.append(i)
        #print(i)
    
    for i in dbcopy:
        charging_pool.append({"evse_id":i['evse_id'],"CPpower":i['powerValue'],"userID":i['chargingInfo']['user_id'],"E":i['chargingInfo']['E'],"T":i['chargingInfo']['T']})
        #print(charging_pool)
    
    for i in charging_pool:
        i['F_t'] = (i['T'] - (i['E']/i['CPpower'])*60)
        if (i['F_t']>= timestep):
            i['F_p'] = i['CPpower']
        else:
            i['F_p'] = (i['F_t'] * i['CPpower']/timestep)
        
        totalPowerDemand += i['CPpower']
        flex_supply += i['F_p']
        i['flex_rec']= 0 #flexibility record
    
    # allocate the charging power again and send back to cpDB
    if totalPowerDemand <= powerLimit:
        #no grid congestion, update the db with full power supply for each CP
        for i in charging_pool:
            newPowerLevel = {"$set":{"updatedPower":i['CPpower']}}
            database_update(i['evse_id'], newPowerLevel)
            
            ### for simulation
            _energy = - i['CPpower']*0.25
            _updatedStatus = {"$inc":{"chargingInfo.E":_energy,'chargingInfo.T': -timestep}}            
            database_update(i['evse_id'], _updatedStatus)
            
        print('all updated1')
        
    else :    
        flex_demand = totalPowerDemand - powerLimit
        #update db with reduced power proportional to each CP, and add flex_rec to each document 
        for i in charging_pool:
            _updatedPower = (i['CPpower'] - flex_demand/flex_supply *i['F_p'])
            _flex = (i['CPpower'] - _updatedPower)*(timestep/60)
            newPowerLevel = {"$set":{"updatedPower":_updatedPower}}
            database_update(i['evse_id'], newPowerLevel)
            newFlex = {"$inc":{"chargingInfo.flex_rec":_flex}}
            database_update(i['evse_id'], newFlex)
            
            ### for simulation
            _energy = - _updatedPower*0.25
            _updatedStatus = {"$inc":{"chargingInfo.E":_energy,'chargingInfo.T': -timestep}}            
            database_update(i['evse_id'], _updatedStatus)    
            
        
        print('all updated2')

In [42]:
for t in range(0,20):
    print("round %d " %t)
    optimization(powerLimit)

round 0 
all updated2
round 1 
all updated2
round 2 
all updated2
round 3 
all updated2
round 4 
all updated2
round 5 
all updated2
round 6 
all updated1
round 7 
all updated1
round 8 
all updated1
round 9 
all updated1
round 10 
all updated1
round 11 
all updated1
round 12 
all updated1
round 13 
all updated1
round 14 
all updated1
round 15 
all updated1
round 16 
all updated1
round 17 
all updated1
round 18 
all updated1
round 19 
all updated1


0.0
0.0
0.0


TypeError: '<' not supported between instances of 'NoneType' and 'float'